# 영업팀 일일보고 DB 저장 프로그램

In [1]:
import pandas as pd
import xlwings as xw
import sys
import pyautogui

In [2]:
# app = xw.App(visible=True) # 실행과정 보이게
app = xw.App(visible=False) # 실행과정 안보이게

## 일일보고 엑셀파일 읽어오기

In [3]:
try:
    wb = xw.Book("./'일일보고.xlsx") # type: xlwings.main.Book
    # sht = wb.sheets[-1] # type: xlwings.main.Sheet
except FileNotFoundError:
    print("같은 폴더안에 \n'일일보고.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    pyautogui.alert(title='FileNotFoundError', text="같은 폴더안에 \n'일일보고.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    sys.exit()

### 시즌별 시트범위 계산

In [4]:
# 시즌
seasons: list = ['22N', '23N', '22S', '23S', '21F', '22F']

# 시트범위 기록할 딕셔너리
season_sheet_range: dict = {}

# 시즌별 시트범위 찾는다.
for i, sheet in enumerate(wb.sheets):
    for season in seasons:
        # season_cleanup = list(set(' '.join(sheet['B4:M4'].options(numbers=str, empty='').value).replace("  ", " ").replace("'", "").replace("+", " ").split(' ')))
        season_cleanup = ''.join(sheet['B4:M4'].options(numbers=str, empty='').value).replace("'", "").replace("+", " ").replace(" ", "")
        season_cleanup = list(set(list(map(''.join, zip(*[iter(season_cleanup)]*3)))))
        if season in season_cleanup:
            if season_sheet_range.get(season + '_start') == None:
                season_sheet_range[season + '_start'] = [sheet.name, i]

# 마지막 시트는 아래처럼 찾는다.
# season_sheet_range['22N_start'][1]

season_sheet_range

{'22N_start': ['210304', 1292],
 '21F_start': ['210304', 1292],
 '22S_start': ['210603', 1353],
 '23N_start': ['220310', 1542],
 '22F_start': ['220310', 1542],
 '23S_start': ['220608', 1603]}

### 좌표 잡는 함수 (상대좌표)

In [5]:
def SetPos(sheet) -> list:

    # 범위 안에 원하는 TEXT 값 찾기
    i: int = 0
    set_position: list = [] # 가변 좌표

    for row in range(200, 1, -1): # row 범위
        for col in range(1, 2): # column 범위
            if sheet.range((row, col)).value == "서울":
                i += 1
                if i < 4: # 뒤에서부터 1, 2, 3번째를 찾는다.
                    # print(f'{col}열 {row}행')
                    set_position.append(row)
                    # print(set_position)

    if len(set_position) != 3:
        print(f"기준좌표 갯수가 3 이어야 합니다.\n현재값 : {len(set_position)}\n\n프로그램을 종료합니다.")
        pyautogui.alert(title='논리 에러', text=f"기준좌표 갯수가 3 이어야 합니다.\n현재값 : {len(set_position)}\n\n프로그램을 종료합니다.")
        sys.exit()
    
    return set_position

### 시트에서 데이터 가져오는 함수

In [6]:
def get_data_from_sheet(sht) -> pd.DataFrame:
    # 시즌 찾기
    # season_cleanup: list = list(set(' '.join(sheet['B4:M4'].options(numbers=str, empty='').value).replace("  ", " ").replace("'", "").replace("+", " ").split(' ')))
    
    # 테이터 기록이 균일하지 못해 어쩔 수 없이 지저분하게 문자열 처리
    # 데이터 변환 과정
    # 리스트 -> 문자열 -> 특문 제거 및 공백없는 시즌만 남은 문자열로 변환 ->
    # 3글자마다 반복해서 잘라냄 -> 리스트 -> 세트(중복삭제) -> 리스트 ->
    # -> 정렬 (시즌 순서는 관계없으나 동일 계열이면 오름차순이어야 한다.)
    
    season_cleanup = ''.join(sheet['B4:M4'].options(numbers=str, empty='').value).replace("'", "").replace("+", " ").replace(" ", "")
    season_cleanup = list(set(list(map(''.join, zip(*[iter(season_cleanup)]*3)))))
    season_cleanup.sort()
    
    N = ''
    S = ''
    F = ''

    for season in season_cleanup:
        # print(season[-1])
        if season[-1] == 'N':
            N = season
        elif season[-1] == 'S':
            S = season
        elif season[-1] == 'F':
            F = season
            
    # 데이터 좌표 찾기
    position: list = SetPos(sht)
    position.sort()
    
    # FN, S, F 시즌 가져오기
    df_FN = sht.range(f'A{position[0]}:C{position[0]+5}').options(pd.DataFrame, index=False, header=False, empty=0).value
    df_S = sht.range(f'A{position[1]}:C{position[1]+5}').options(pd.DataFrame, index=False, header=False, empty=0).value
    df_F = sht.range(f'A{position[2]}:C{position[2]+5}').options(pd.DataFrame, index=False, header=False, empty=0).value

    # N시즌 구하기
    df_N = df_FN.copy()
    df_N.iloc[:, 1:3] = df_FN.iloc[:, 1:3] - df_F.iloc[:, 1:3]

    # 컬럼 정리
    df_FN.columns = ['상권', '수주량', '해제량']
    df_N.columns = ['상권', '수주량', '해제량']
    df_S.columns = ['상권', '수주량', '해제량']
    df_F.columns = ['상권', '수주량', '해제량']

    # 데이터타입 정리
    df_FN.iloc[:, 1:3] = df_FN.iloc[:, 1:3].astype(int)
    df_N.iloc[:, 1:3] = df_N.iloc[:, 1:3].astype(int)
    df_S.iloc[:, 1:3] = df_S.iloc[:, 1:3].astype(int)
    df_F.iloc[:, 1:3] = df_F.iloc[:, 1:3].astype(int)

    # 보고일자 추가
    df_FN['보고일자'] = '20' + sht.name
    df_FN['보고일자'] = pd.to_datetime(df_FN['보고일자'])
    df_N['보고일자'] = '20' + sht.name
    df_N['보고일자'] = pd.to_datetime(df_N['보고일자'])
    df_S['보고일자'] = '20' + sht.name
    df_S['보고일자'] = pd.to_datetime(df_S['보고일자'])
    df_F['보고일자'] = '20' + sht.name
    df_F['보고일자'] = pd.to_datetime(df_F['보고일자'])

    # 시즌 컬럼 추가
    df_FN['시즌'] = F + N
    df_N['시즌'] = N
    df_S['시즌'] = S
    df_F['시즌'] = F

    df = pd.concat([df_N, df_F, df_S])
    
    return df

### 시트 순회

In [ ]:
df_total = pd.DataFrame()

for j, sheet in enumerate(wb.sheets):
    if j < 1292: # 슬라이싱해서 순회할 방법을 찾지 못해 전체순회 돌리고 앞쪽 1290여개 시트는 패스 (총 1700여개. 실제 순회는 400개정도)
        continue
    # print(f'순번 {j}, {sheet.name} 시트 진행중입니다.')
    
    df_sheet = get_data_from_sheet(sheet)
    
    df_total = pd.concat([df_total, df_sheet])

In [ ]:
app.kill()

In [ ]:
df_total

In [ ]:
df_total.groupby(['시즌', '상권'])[['수주량', '해제량']].agg(sum)

## SQLITE 기록

- DB명 : daliy_order.db
- 테이블명 : SALES_SUJU_HAJE_T
- 옵션 : 테이블이 존재할 경우 삭제 후 재기록

In [ ]:
import sqlite3
connect = sqlite3.connect('./data/daliy_order.db') # conn 객체 생성 = DB 파일명
cursor = connect.cursor() # 커서 생성
df_total.to_sql('SALES_SUJU_HAJE_T', connect, index=False, if_exists='replace') # 테이블이 존재할 경우 데이터만 넣는다.
connect.close()